In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
cd /mnt/RILD/RILD_pipeline

/mnt/RILD/RILD_pipeline


In [3]:
pip install -e .

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///mnt/RILD/RILD_pipeline
  Preparing metadata (setup.py) ... done
  Running setup.py develop for RILD-pipeline
Note: you may need to restart the kernel to use updated packages.


importing the RILD_pipeline module functions/constructs

In [13]:
from RILD_pipeline import format
from RILD_pipeline.operations import parameters
from RILD_pipeline.operations.Patient import Patient
from RILD_pipeline import transform
from RILD_pipeline import classify_tissues
from RILD_pipeline import register
from RILD_pipeline import dose_per_class

Setting the data directory, then creating the Patient object and parameters object

In [8]:
data_dir = '/mnt/RILD/data/'

In [9]:
params = parameters(params_dict = {'orig_image_folder': 'orig', 'aligned_image_folder' : 'aligned', 
                                   'inference': {'nr_class_inference':6, 'nr_fold_eval':19}, 'format' : {'M' : 1, 'sigma' : 1}, 
                                   'models' : {'format.segment': { 'bones':'segmentation/skeleton_best_metric_model.pth', 
                                                                 'airways': 'segmentation/airway_best_metric_model.pth',
                                                                  'lungs': 'segmentation/lungs_best_metric_model.pth'} , 
                                   'classify_tissues' : ['classification/UNet_drop00_T5_CE_old_loss_monai_plus_def_proper_norm_classes6_pretrained_new_net_False_rotTrue_int_augTrue_reduce0_f32_lr0.0001_ep100_b16_best_model.pt',
                                                                        'classification/UNet_drop00_T5_CE_old_loss_monai_plus_def_proper_norm_classes6_pretrained_new_net_False_rotTrue_int_augTrue_reduce0_f64_lr0.0001_ep100_b8_best_model.pt', 
                                                                        'classification/UNet_drop00_T5_DC_old_loss_monai_plus_def_proper_norm_classes6_pretrained_new_net_False_rotTrue_int_augTrue_reduce0_f32_lr0.0001_ep100_b16_best_model.pt',
                                                                        'classification/UNet_drop00_T5_DC_old_loss_monai_plus_def_proper_norm_classes6_pretrained_new_net_False_rotTrue_int_augTrue_reduce0_f64_lr0.0001_ep100_b8_best_model.pt',
                                                                        'classification/UNet_drop00_T5_LV_old_loss_monai_plus_def_proper_norm_classes6_pretrained_new_net_False_rotTrue_int_augTrue_reduce0_f32_lr0.0001_ep100_b16_best_model.pt',
                                                                       'classification/UNet_drop00_T5_LV_old_loss_monai_plus_def_proper_norm_classes6_pretrained_new_net_False_rotTrue_int_augTrue_reduce0_f64_lr0.0001_ep100_b8_best_model.pt']

   }})
               

In [10]:
rild_patient = Patient('SABR0195','orig', 'aligned', params, create_log_file = True, patients_data_dir = data_dir)

2023-10-18 10:12:44,287 [INFO] searching for models for format.segment
2023-10-18 10:12:44,293 [INFO] For function format.segment model in path /mnt/RILD/data/models/segmentation/skeleton_best_metric_model.pth exists.
2023-10-18 10:12:44,296 [INFO] For function format.segment model in path /mnt/RILD/data/models/segmentation/airway_best_metric_model.pth exists.
2023-10-18 10:12:44,299 [INFO] For function format.segment model in path /mnt/RILD/data/models/segmentation/lungs_best_metric_model.pth exists.
2023-10-18 10:12:44,303 [INFO] searching for models for classify_tissues
2023-10-18 10:12:44,308 [INFO] For function classify_tissues model in path /mnt/RILD/data/models/classification/UNet_drop00_T5_CE_old_loss_monai_plus_def_proper_norm_classes6_pretrained_new_net_False_rotTrue_int_augTrue_reduce0_f32_lr0.0001_ep100_b16_best_model.pt exists.
2023-10-18 10:12:44,311 [INFO] For function classify_tissues model in path /mnt/RILD/data/models/classification/UNet_drop00_T5_CE_old_loss_monai_pl

{'main_dir': '/mnt/RILD/data/SABR0195', 'orig_image': '/mnt/RILD/data/SABR0195/DICOM', 'models_path': '/mnt/RILD/data/models', 'radio_image': '/mnt/RILD/data/SABR0195/radiotherapy/', 'reg_path': '/mnt/RILD/data/SABR0195/registrations/results/', 'aff_path': '/mnt/RILD/data/SABR0195/affine', 'tissue_classes': '/mnt/RILD/data/SABR0195/tissue_classes', 'orig_base_image': '/mnt/RILD/data/SABR0195/CT/orig', 'orig_seg_image': '/mnt/RILD/data/SABR0195/segmentations/orig', 'orig_stack_path': '/mnt/RILD/data/SABR0195/stacks/orig', 'orig_radio_image': '/mnt/RILD/data/SABR0195/radiotherapy/orig', 'aligned_base_image': '/mnt/RILD/data/SABR0195/CT/aligned', 'aligned_seg_image': '/mnt/RILD/data/SABR0195/segmentations/aligned', 'aligned_stack_path': '/mnt/RILD/data/SABR0195/stacks/aligned', 'aligned_radio_image': '/mnt/RILD/data/SABR0195/radiotherapy/aligned'}


First we convert to nifti

In [10]:
#First convert the dicom files to nifti
format.convert_dcm2nii(rild_patient, params)

In [11]:
#Can also try converting RT struct files but doesn't always work
format.convert_rt(rild_patient)

Now we can start segmenting

In [12]:
format.segment(rild_patient, params, "airways")

2023-10-04 12:18:03,204 [INFO] Initialising the airways segmentation
Loading dataset: 100%|████████████████████████████| 5/5 [00:04<00:00,  1.04it/s]


file written: /mnt/RILD/data/benchmarkIDEAL/lung062/segmentations/orig/obs0_airways.nii.gz.
file written: /mnt/RILD/data/benchmarkIDEAL/lung062/segmentations/orig/obs1_airways.nii.gz.
file written: /mnt/RILD/data/benchmarkIDEAL/lung062/segmentations/orig/obs2_airways.nii.gz.
file written: /mnt/RILD/data/benchmarkIDEAL/lung062/segmentations/orig/obs3_airways.nii.gz.


2023-10-04 12:19:09,712 [INFO] airways segmentation is done


file written: /mnt/RILD/data/benchmarkIDEAL/lung062/segmentations/orig/obs4_airways.nii.gz.


In [13]:
format.segment(rild_patient, params, "lungs")

2023-10-04 12:23:16,907 [INFO] Initialising the lungs segmentation
Loading dataset: 100%|████████████████████████████| 5/5 [00:03<00:00,  1.25it/s]


file written: /mnt/RILD/data/benchmarkIDEAL/lung062/segmentations/orig/obs0_lungs.nii.gz.
file written: /mnt/RILD/data/benchmarkIDEAL/lung062/segmentations/orig/obs1_lungs.nii.gz.
file written: /mnt/RILD/data/benchmarkIDEAL/lung062/segmentations/orig/obs2_lungs.nii.gz.
file written: /mnt/RILD/data/benchmarkIDEAL/lung062/segmentations/orig/obs3_lungs.nii.gz.
file written: /mnt/RILD/data/benchmarkIDEAL/lung062/segmentations/orig/obs4_lungs.nii.gz.


2023-10-04 12:24:38,448 [INFO] lungs segmentation is done


In [ ]:
format.segment(rild_patient, params, "bones")

In [ ]:
format.segment_vessels(rild_patient)

This is the initial alignment. Change bone_reg to False if you want it to use the image intensity information to align the images.

In [ ]:
format.register_bones(rild_patient, params, bone_reg=True)

Here we generate the features for the deformable registration between timepoints, and then we run the registration.

In [ ]:
transform.transform(rild_patient)

In [ ]:
register.register(rild_patient)

Here we do the tissue classification. This step is independent of the registration step before it, all you need as a prerequisite is the aligned CTs and lung masks.

In [ ]:
classify_tissues.classify_tissues(rild_patient, params)

This is the dose-tissue relationship step. All the previous parts of the pipeline need to have results for this to run.

In [ ]:
dose_per_class.dose_per_class(rild_patient)